In [1]:
import logging
import pandas as pd
import datetime
import seaborn as sb
import time
import os.path
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CovidDatasets
from libs.CovidUtil import CovidUtil
pd.set_option('display.float_format', lambda x: '%.12f' % x)

In [2]:
def record_results(res, directory, name, num, pop):
    import copy
    import os.path
    vals = copy.copy(res)
    # Format the date in the manner the front-end expects
    vals['Date'] = res['Date'].apply(lambda d: "{}/{}/{}".format(d.month, d.day, d.year)) 
    # Set the population
    vals['Population'] = pop
    # Write the results to the specified directory
    with open( os.path.join(directory, name + '.' + str(num) + '.json').format(name), 'w') as out:
        out.write(
            str(vals[[
                'Date', 
                'R', 
                'Beg. Susceptible', 
                'New Inf.', 
                'Curr. Inf.', 
                'Recov. or Died', 
                'End Susceptible', 
                'Actual Reported', 
                'Pred. Hosp.', 
                'Cum. Inf.', 
                'Cum. Deaths', 
                'Avail. Hosp. Beds', 
                'S&P 500', 
                'Est. Actual Chance of Inf.', 
                'Pred. Chance of Inf.', 
                'Cum. Pred. Chance of Inf.', 
                'Population',
                'R0', 
                '% Susceptible'
            ]].values.tolist())
        )

In [6]:
def model_state(country, state, interventions=None):
    ## Constants
    start_time = time.time()
    HOSPITALIZATION_RATE = .0727
    HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
    TOTAL_INFECTED_PERIOD = 12
    MODEL_INTERVAL = 4
    r0 = 2.4
    Dataset = CovidDatasets()
    POP = Dataset.get_population_by_country_state(country, state)
    # Pack all of the assumptions and parameters into a dict that can be passed into the model
    MODEL_PARAMETERS = {
        # Pack the changeable model parameters
        'timeseries': Dataset.get_timeseries_by_country_state(country, state, MODEL_INTERVAL),
        'beds': Dataset.get_beds_by_country_state(country, state),
        'population': POP,
        'projection_iterations': 25, # Number of iterations into the future to project
        'r0': r0,
        'interventions': interventions,
        'hospitalization_rate': HOSPITALIZATION_RATE,
        'initial_hospitalization_rate': .05,
        'case_fatality_rate': .0109341104294479,
        'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        # Assumes that anyone who needs ICU care and doesn't get it dies
        'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
        'hospital_capacity_change_daily_rate': 1.05,
        'max_hospital_capacity_factor': 2.07,
        'initial_hospital_bed_utilization': .6,
        'model_interval': 4, # In days
        'total_infected_period': 12, # In days
        'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
    }
    return CovidTimeseriesModel().forecast(model_parameters=MODEL_PARAMETERS)

r0 = 2.4

INTERVENTIONS = [
    None, 
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.7,
        datetime.date(2020, 6, 23): r0
    },
    {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 3, 31): 0.3,
        datetime.date(2020, 4, 28): 0.2,
        datetime.date(2020, 5,  6): 0.1,
        datetime.date(2020, 5, 10): 0.35,
        datetime.date(2020, 5, 18): r0
    }
]

model_state('USA', 'CA', INTERVENTIONS[0])

,Note,Date,Timestamp,R,Effective R.,Beg. Susceptible,New Inf.,Curr. Inf.,Recov. or Died,End Susceptible,...,Pred. Hosp.,Cum. Inf.,Cum. Deaths,Avail. Hosp. Beds,S&P 500,Est. Actual Chance of Inf.,Pred. Chance of Inf.,Cum. Pred. Chance of Inf.,R0,% Susceptible
0,,2020-03-03,1583226000.000000000000,2.400000000000,nan,39511879.121045395732,343.878954607978,0.000000000000,0.000000000000,39511879.121045395732,...,0.000000000000,nan,nan,28448.800000000003,None,0.000000000000,None,None,None,None
1,,2020-03-07,1583571600.000000000000,3.240000000000,3.239971801946,39510764.962929181755,1114.158116212386,343.878954607978,0.000000000000,39510764.962929181755,...,80.999295048640,1114.158116212386,12.182327878532,28448.800000000003,None,0.000056396109,None,None,None,None
2,,2020-03-11,1583913600.000000000000,2.185185185185,2.185104549859,39508330.410960182548,2434.551968997545,1458.037070820364,0.000000000000,39508330.410960182548,...,176.991928146122,3548.710085209931,38.801987953781,28448.800000000003,None,0.000123235941,None,None,None,None
3,,2020-03-15,1584259200.000000000000,2.406779661017,2.406542554537,39502471.558045558631,5858.852914623250,3892.589039817909,0.000000000000,39502471.558045558631,...,425.938606893110,9407.562999833182,102.863332712164,29871.240000000005,None,0.000296601757,None,None,None,None
4,,2020-03-19,1584604800.000000000000,2.105633802817,2.105114141703,39490138.003920823336,12333.554124732937,9407.562999833182,343.878954607978,39490138.003920823336,...,896.649384868084,21741.117124566117,237.719775499567,31364.802000000007,None,0.000624534686,None,None,None,None
5,,2020-03-23,1584950400.000000000000,2.439241917503,2.437878525517,39460070.297176837921,30067.706743986364,20626.959008353733,1458.037070820364,39460070.297176837921,...,2185.922280287808,51808.823868552485,566.483401398569,32933.042100000006,None,0.001523391185,None,None,None,None
6,,2020-03-27,1585296000.000000000000,2.400000000000,2.396832208434,39388003.049219109118,72067.247957732980,48260.113783342546,3892.589039817909,39388003.049219109118,...,5239.288926527188,123876.071826285464,1354.474648914825,34579.694205000007,None,nan,None,None,None,None
7,,2020-03-31,1585641600.000000000000,2.400000000000,2.392454793498,39215585.416388392448,172417.632830712624,114468.508826452278,9751.441954441159,39215585.416388392448,...,12534.761906792808,296293.704656998103,3239.708086269839,36308.678915250006,None,nan,None,None,None,None
8,,2020-04-04,1585987200.000000000000,2.400000000000,2.381982026153,38804889.713993862271,410695.702394530585,274552.587532431935,22084.996079174096,38804889.713993862271,...,29857.577564082374,706989.407051528688,7730.300249151306,38124.112861012509,None,nan,None,None,None,None
9,,2020-04-08,1586332800.000000000000,2.400000000000,2.357036082571,37836865.124493181705,968024.589500682079,655180.583182976115,52152.702823160464,37836865.124493181705,...,70375.387656699590,1675013.996552210767,28146.229664813669,40030.318504063136,None,nan,None,None,None,None
